In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import torchvision

In [7]:
# Configuração das transformações de imagem
transform = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Carregando o dataset (substitua 'path_to_data' pelo caminho do seu dataset)
dataset = datasets.ImageFolder(root='/home/rodrigo/Repos/Hackathon-NTT/src/modelos/dataset/images/', transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4)


In [8]:
class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, img_dim),
            nn.Tanh(),
        )

    def forward(self, x):
        return self.gen(x)

class Discriminator(nn.Module):
    def __init__(self, img_dim):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Linear(img_dim, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Garantir que a entrada seja achatada corretamente
        return self.disc(x)


def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)


# Definindo parâmetros do modelo
z_dim = 100
img_dim = 128 * 128 * 3  # Para imagens de 128x128 com 3 canais de cor
lr = 0.0002
batch_size = 64
num_epochs = 220

# Criando instâncias dos modelos
gen = Generator(z_dim, img_dim)
disc = Discriminator(img_dim)
initialize_weights(gen)
initialize_weights(disc)

# Configuração do otimizador e da função de perda
opt_gen = optim.Adam(gen.parameters(), lr=lr)
opt_disc = optim.Adam(disc.parameters(), lr=lr)
criterion = nn.BCELoss()

# Movendo modelos para GPU se disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gen.to(device)
disc.to(device)


Discriminator(
  (disc): Sequential(
    (0): Linear(in_features=49152, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
    (6): Linear(in_features=256, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [9]:
import os

# Diretório onde as imagens serão salvas
output_dir = '/home/rodrigo/Repos/Hackathon-NTT/src/modelos/images/'

# Verifica se o diretório existe, se não existir, cria o diretório
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

fixed_noise = torch.randn(batch_size, z_dim).to(device)

for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(dataloader):
        real = real.view(-1, img_dim).to(device)
        
        # Gerar imagens falsas usando o gerador
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)

        # Treinando o discriminador
        disc_real = disc(real).view(-1)
        loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake.detach()).view(-1)
        loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        loss_disc = (loss_disc_real + loss_disc_fake) / 2
        disc.zero_grad()
        loss_disc.backward()
        opt_disc.step()

        # Treinando o gerador
        output = disc(fake).view(-1)
        loss_gen = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

    print(f"Epoch [{epoch}/{num_epochs}]  Loss D: {loss_disc:.4f}, Loss G: {loss_gen:.4f}")

    if epoch % 10 == 0:
        with torch.no_grad():
            fake = gen(fixed_noise).reshape(-1, 3, 128, 128)
            img_grid = torchvision.utils.make_grid(fake, normalize=True)
            torchvision.utils.save_image(img_grid, os.path.join(output_dir, f"epoch_{epoch}.png"))


Epoch [0/220]  Loss D: 0.0018, Loss G: 7.7455
Epoch [1/220]  Loss D: 0.0509, Loss G: 10.7221
Epoch [2/220]  Loss D: 0.0102, Loss G: 6.0236
Epoch [3/220]  Loss D: 0.0492, Loss G: 6.4831
Epoch [4/220]  Loss D: 0.0656, Loss G: 3.9147
Epoch [5/220]  Loss D: 0.2319, Loss G: 3.2086
Epoch [6/220]  Loss D: 0.0659, Loss G: 6.4550
Epoch [7/220]  Loss D: 25.0000, Loss G: 100.0000
Epoch [8/220]  Loss D: 50.0000, Loss G: 0.0000
Epoch [9/220]  Loss D: 50.0000, Loss G: 0.0000
Epoch [10/220]  Loss D: 50.0000, Loss G: 0.0000
Epoch [11/220]  Loss D: 50.0000, Loss G: 0.0000
Epoch [12/220]  Loss D: 50.0000, Loss G: 0.0000
Epoch [13/220]  Loss D: 50.0000, Loss G: 0.0000
Epoch [14/220]  Loss D: 50.0000, Loss G: 0.0000
Epoch [15/220]  Loss D: 50.0000, Loss G: 0.0000
Epoch [16/220]  Loss D: 50.0000, Loss G: 0.0000
Epoch [17/220]  Loss D: 50.0000, Loss G: 0.0000
Epoch [18/220]  Loss D: 50.0000, Loss G: 0.0000
Epoch [19/220]  Loss D: 50.0000, Loss G: 0.0000
Epoch [20/220]  Loss D: 50.0000, Loss G: 0.0000
Epoch 

KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    noise = torch.randn(64, z_dim).to(device)
    fake_images = gen(noise).reshape(-1, 3, 128, 128)
    torchvision.utils.save_image(fake_images, "generated_images.png", normalize=True)
